## chapter11. 심층신경망 훈련
### 11.1. 그래디언트 소실과 폭주문제
- gradient descent 훈련시 나타날 수 있는 문제점
1. gradient vanishing : 알고리즘이 하위층으로 진행됨에 따라 그래디언트는 점차 작아져 하위층의 연결가중치를 실제 반영하지 못하는 경우 
2. graidient exploding : 그래디언트가 점차 커져 여러개의 층이 비정상적으로 큰 가중치로 갱신되는 경우를 의미한다. 
- 원인 : gradient는 결국 미분값(=변화량)을 의미한다. 이 변화량이 작다면 network를 효과적으로 학습하지 못하게 되고, error-rate가 다 낮아지기 전에 다른 값으로 수렴하게 된다.   (ex) sigmoid에서 0이나 1로 수렴하는 경우 
- 활성함수에서 0이나 1값으로 근사되어 버려, 초기 레이어에서 parameter-value의 큰 변화가 발생해도 output의 영향이 없어지는 문제점이다. (수식에 대한 관점의 설명은 아래의 bolg를 참고한다)

- vanishing gradient, exploding gradient 설명 blog(수식참고) : https://ratsgo.github.io/deep%20learning/2017/10/10/RNNsty/
- solution blog(필수) : https://wikidocs.net/61375
- 기타 한국어 참고 설명: https://ydseo.tistory.com/41


## 해결방안
- 가중치 초기화 (세이비어 초기화, He초기화)
- 수렴하지 않는 활성함수의 사용(ReLU, ReLU변형함수들)
- 배치정규화(배치정규-> 결괏값 스케일 조정 후 이동)
- 그래디언트 클리핑

<br>

### 11.1.1. 세이비어 초기화와 He초기화 (가중치 초기화 방법) 
- Xavier initializer : 시그모이드, 하이퍼볼릭 탄젠트 함수를 활성함수로 사용할 때 사용하는 가중치 초기화 방법. (논문: http://proceedings.mlr.press/v9/glorot10a/glorot10a.pdf)
1. 균등분포로 초기화 : W~uni(-$sqrt(6/n_in+n_out)$, +$sqrt(6/n_in+n_out)$)
2. 정규분포로 초기화 : W~N(0, $2/n_in+n_out$)

<br>

- He initializer : ReLU 함수를 활성함수로 사용할 때 사용하는 가중치 초기화 방법. (논문 : https://www.cv-foundation.org/openaccess/content_iccv_2015/papers/He_Delving_Deep_into_ICCV_2015_paper.pdf)
1. 균등분포기반 초기화 :  W~uni(-$sqrt(6/n_in)$, +$sqrt(6/n_in)$)
2. 정규분포기반 초기화 : W~N(0, $2/n_in$)

<br>

- 시그모이드 함수나 하이퍼볼릭탄젬트 함수를 활성함수로 사용할 때는 세이비어 초기화 방법을 사용하는 것이 효율적이다. 
- ReLU함수를 사용할 때는 He초기화 방식이 적절하다. 
- ReLU+He 초기화 방식을 사용하는 것이 일반적이다 


In [5]:
#idea: 완전 연결층을 생성하기 전에 초기화 먼저 시작한다. 
import warnings
warnings.filterwarnings('ignore')
#데이터는 MNIST데이터를 사용한다. 
%tensorflow_version 1.x #버전지정
import tensorflow as tf

reset_graph()
n_inputs=28*28 #뉴런의 수를 지정하는 경우 
n_hidden1=300

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X") #변수지정
#변수 초기화 
he_init=tf.variance_scaling_initializer()
hidden1=tf.layers.dense(X,n_hidden1, activation=tf.nn.relu,
                          kernel_initializer=he_init, name='hidden1', reuse=tf.AUTO_REUSE)

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `1.x #버전지정`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


### 11.1.2. 수렴하지 않는 활성화함수 
- 활성화함수를 잘못사용하면 그래디언트 폭주나 그래디언트 소실 문제로 이어질 수가 있다. 
- 은닉층에서 사용하는 활성화함수를 정할 필요가 있음.(가중치에 영향을 주는 함수는 결국 은닉층에서의 함수이므로)
- 은닉층에서는 sigmoid를 사용하지 않는다
- Leaky ReLU, RReLU, PReLE (변종 ReLU)를 사용하면 죽은 ReLU문제를 해결한다. 
- 혹은 ELU사용도 가능하다.혹은  SELU도 가능(계산속도는 느리다, 하지만 기존의 ReLU와는 다르케 z<0이어도 0으로 수렵하지 않는다. )
- Leaky ReLU 논문 : https://arxiv.org/pdf/1505.00853.pdf
- 은닉층에서 사용하는 활성함수 : ELU > Leaky ReLU > tanh > 로지스틱(시그모이드) 


Leaky ReLU

In [0]:
#활성함수 정의 
def leaky_relu(z, name=None):
    return tf.maximum(0.01 * z, z, name=name)

In [0]:
#그래프 구성
reset_graph()

X=tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
hidden1=tf.layers.dense(X, n_hidden1, activation=leaky_relu, name="hidden1")

In [21]:
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10


X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=leaky_relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=leaky_relu, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [22]:
#데이터를 실제로 넣고 모델을 실제로 실행하는 단계
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

n_epochs = 40
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        if epoch % 5 == 0:
            acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
            acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
            print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)

    save_path = saver.save(sess, "./my_model_final.ckpt")

11493376/11490434 [==============================] - 0s 0us/step
0 Batch accuracy: 0.86 Validation accuracy: 0.904
5 Batch accuracy: 0.94 Validation accuracy: 0.9496
10 Batch accuracy: 0.92 Validation accuracy: 0.9652
15 Batch accuracy: 0.94 Validation accuracy: 0.9706
20 Batch accuracy: 1.0 Validation accuracy: 0.9762
25 Batch accuracy: 1.0 Validation accuracy: 0.9776
30 Batch accuracy: 0.98 Validation accuracy: 0.9782
35 Batch accuracy: 1.0 Validation accuracy: 0.9786


ELU

In [0]:
#은닉층에 ELU사용하는 경우 
#ELU정의 

def elu(z, alpha=1):
  return np.where(z<0, alpha*(np.exp(z)-1),z)

In [0]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")

In [0]:
#elu를 은닉층의 활성함수로 사용하는 경우 
hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.elu, name="hidden1", reuse=tf.AUTO_REUSE)

### 11.1.3. 배치 정규화 
- ReLU함수+He초기화를 사용해도 그래디언트 소실과 폭주가 일어날 수가 있다. 이에 따라 해줘야 하는 작업.
- 배치정규화는 신경망의 각 층에 들어가는 입력을 평균과 분산으로 정규화하여 학습을 효율적으로 만든다.(다만 각 배치샘플에 따라 정규화해줘야 하므로 '배치'정규화라 한다)
- 배치정규화는 '내부공변량의 변화'에 의해 일어나는 현상이다.
1. 활성함수 통화 전에 추가하는 연산(연산의 과정은 아래의 그림 참고)
2. 층의 스케일 파라미터, 층의 이동 파라미터를 조정하여 작성된다. 
- 논문 : https://arxiv.org/pdf/1502.03167v3.pdf

![image](https://user-images.githubusercontent.com/49298791/74802870-682b1680-531e-11ea-80fa-7013bea4885e.png)

In [0]:
#실행하기 전, 그래프 reset하는 방법
import numpy as np
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [7]:
#배치 정규하 과정
reset_graph() 
import tensorflow as tf
import numpy as np

n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")

training = tf.placeholder_with_default(False, shape=(), name='training')

hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
bn1 = tf.layers.batch_normalization(hidden1, training=training, momentum=0.9)
bn1_act = tf.nn.elu(bn1)

hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
bn2 = tf.layers.batch_normalization(hidden2, training=training, momentum=0.9)
bn2_act = tf.nn.elu(bn2)

logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
logits = tf.layers.batch_normalization(logits_before_bn, training=training,
                                       momentum=0.9)

Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).


In [0]:
reset_graph()

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
training = tf.placeholder_with_default(False, shape=(), name='training')

In [0]:
#배치정규화의 매개변수의 반복을 없애기 위한 partial을 사용한다
from functools import partial

my_batch_norm_layer = partial(tf.layers.batch_normalization,
                              training=training, momentum=0.9)

hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
bn1 = my_batch_norm_layer(hidden1) #단순한 배치 정규화를 partial로 저정
bn1_act = tf.nn.elu(bn1)

hidden2 = tf.layers.dense(bn1_act, n_hidden2, name="hidden2")
bn2 = my_batch_norm_layer(hidden2)
bn2_act = tf.nn.elu(bn2)

#출력층
logits_before_bn = tf.layers.dense(bn2_act, n_outputs, name="outputs")
logits = my_batch_norm_layer(logits_before_bn)

In [12]:
#BN 실행시 필요한 shuffle함수 
reset_graph()
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

#데이터 로드 
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=leaky_relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=leaky_relu, name="hidden2")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


```python
#실행단계
#각 실행의 데이터마다 training의 placeholder를 사용해야 한다. 
n_epochs = 20
batch_size = 200

extra_update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run([training_op, extra_update_ops],
                     feed_dict={training: True, X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "my_model_final.ckpt")
```

또한 비용을 최소화하는 파라미터를 찾을 수 있도록 saver를 지정하여 파라미터를 사용할 수가 있다. 



### 11.1.4. 그래디언트 클리핑
- gradient의 최대갯수를 조정하고 graident가 최대치(threshold)를 넘게 되면 gradient의 크기를 재조정하여 gradient의 크기를 유지하는 방법이다. 
- 업데이트 하는 방법은 아래와 같다. 
- 논문 : http://proceedings.mlr.press/v28/pascanu13.pdf


![image](https://user-images.githubusercontent.com/49298791/74804869-d83c9b00-5324-11ea-99c6-72abc53e226a.png)

그래디언트 클리핑 구성단계

In [0]:
#기본적인 layer구성에 대한 정의 
reset_graph()

n_inputs = 28 * 28  # MNIST
n_hidden1 = 300
n_hidden2 = 50
n_hidden3 = 50
n_hidden4 = 50
n_hidden5 = 50
n_outputs = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, activation=tf.nn.relu, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
    hidden3 = tf.layers.dense(hidden2, n_hidden3, activation=tf.nn.relu, name="hidden3")
    hidden4 = tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, name="hidden4")
    hidden5 = tf.layers.dense(hidden4, n_hidden5, activation=tf.nn.relu, name="hidden5")
    logits = tf.layers.dense(hidden5, n_outputs, name="outputs")

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

In [0]:
threshold=1.0 #하이퍼파라미터 튜닝 대상
learning_rate=0.01

optimizer= tf.train.GradientDescentOptimizer(learning_rate)
grads_and_vars=optimizer.compute_gradients(loss) #비용함수 고려 
capped_gvs=[(tf.clip_by_value(grad, -threshold, threshold), var)
for grad, var in grads_and_vars]
training_ops=optimizer.apply_gradients(capped_gvs)

그래디언트 클리핑 실행단계

In [0]:
#평가 
with tf.name_scope("eval"):
  correct=tf.nn.in_top_k(logits,y,1) #top값을 가지는 경우 
  accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")

In [0]:
#실행단계 변수 초기화 
init=tf.global_variables_initializer()
saver=tf.train.Saver()

In [0]:
reset_graph()

batch_norm_momentum = 0.9

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int32, shape=(None), name="y")
training = tf.placeholder_with_default(False, shape=(), name='training')

with tf.name_scope("dnn"):
    he_init = tf.variance_scaling_initializer()

    my_batch_norm_layer = partial(
            tf.layers.batch_normalization,
            training=training,
            momentum=batch_norm_momentum)

    my_dense_layer = partial(
            tf.layers.dense,
            kernel_initializer=he_init)

    hidden1 = my_dense_layer(X, n_hidden1, name="hidden1")
    bn1 = tf.nn.elu(my_batch_norm_layer(hidden1))
    hidden2 = my_dense_layer(bn1, n_hidden2, name="hidden2")
    bn2 = tf.nn.elu(my_batch_norm_layer(hidden2))
    logits_before_bn = my_dense_layer(bn2, n_outputs, name="outputs")
    logits = my_batch_norm_layer(logits_before_bn)

with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [0]:
n_epochs=20
batch_size=200

In [0]:
#세션 실행
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        accuracy_val = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Validation accuracy:", accuracy_val)

    save_path = saver.save(sess, "./my_model_final.ckpt")

### 11.2, 미리 훈련된 층 재사용하기 
- **전이학습** : 해결하려는 것과 비슷한 유형의 문제를 처리한 신경망이 있는지를 찾아보고 그 신경망의 하위층을 재사용하는 것. 훈련속도를 높이고 필요한 훈련데이터도 적다. 
- 지도학습의 경우, 전이학습의 방법은 기존의 학습된 모델에서 마지막 층을 제거한 후에 분류하고자 하는 층을 연결시켜주고 학습시킨다. 
- 전이 가능한 문제의 데이터는 많은데 전이 하려고 하는 문제의 데이터가 적을 때, 입력데이터가 같고 저레벨 특성의 문제를 해결하는데 도움이 될 때 사용한다. 
- (단 원래 사용한 것과 크기가 다른 이미지를 입력으로 사용한다면 원본 모델에 맞는 크기로 변경하는 전처리 단계를 거칠 필요가 있다. )


#### 11.2.1. 텐서플로 모델 재사용하기

In [17]:
%tensorflow_version 1.x #버전지정
import tensorflow as tf
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

`%tensorflow_version` only switches the major version: `1.x` or `2.x`.
You set: `1.x #버전지정`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


```python
#앞에서 사용했던 그래프 그대로 가져온다
reset_graph()
saver=tf.train.import_meta_graph("./my_model_final.ckpt.meta")

#훈련 대상인 텐서를 직접 지정한다
X=tf.get_default_graph.get_tensor_by_name("X:0")
y=tf.get_default_graph.get_tensor_by_name("y:0")
accuracy=tf.get_default_graph().get_tensor_by_name("eval/accuracy:0")
training_op=tf.get_default_graph().get_operation_by_name("GradientDescent")
```

In [30]:
#지금까지의 모든 연결된 연산리스트 확인
for op in tf.get_default_graph().get_operations():
    print(op.name)

X
y
hidden1/kernel/Initializer/random_uniform/shape
hidden1/kernel/Initializer/random_uniform/min
hidden1/kernel/Initializer/random_uniform/max
hidden1/kernel/Initializer/random_uniform/RandomUniform
hidden1/kernel/Initializer/random_uniform/sub
hidden1/kernel/Initializer/random_uniform/mul
hidden1/kernel/Initializer/random_uniform
hidden1/kernel
hidden1/kernel/Assign
hidden1/kernel/read
hidden1/bias/Initializer/zeros
hidden1/bias
hidden1/bias/Assign
hidden1/bias/read
dnn/hidden1/MatMul
dnn/hidden1/BiasAdd
dnn/hidden1/mul/x
dnn/hidden1/mul
dnn/hidden1/Maximum
hidden2/kernel/Initializer/random_uniform/shape
hidden2/kernel/Initializer/random_uniform/min
hidden2/kernel/Initializer/random_uniform/max
hidden2/kernel/Initializer/random_uniform/RandomUniform
hidden2/kernel/Initializer/random_uniform/sub
hidden2/kernel/Initializer/random_uniform/mul
hidden2/kernel/Initializer/random_uniform
hidden2/kernel
hidden2/kernel/Assign
hidden2/kernel/read
hidden2/bias/Initializer/zeros
hidden2/bias
hid

```python
#원본 모델에 대하여 문서화한다. 
for op in (X,y,accuracy, training_op):
  tf.add_to_collection("my_training_op", op)
#위와 같이 저장한 모델을 재사용할 때는 
X,y,accuracy,training_op=tf.get_collection("my_training_op")

#미리 훈련된 층 위에 새로운 모델을 훈련하는 경우 
#출력에 대한 손실을 계산하고 이를 최소화하기 위한 옵티마이저의 사용
reuse_vars=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope="hidden[123]")
restore_saver=tf.train.Saver(reuse_vars) #복원
init=tf.global_variables_initializer() #변수 초기화 
saver=tf.train.Saver() #새로운 모델저장

#실행
with tf.Session() as sess:
  init.run()
  restore_saver.restore(sess,"./my_model_final.ckpt")
  save_path=sess.saver(sess,"./my_model_final.ckpt")
```


#### 11.2.2.다른 프레임워크 모델 재사용하기 
- 만약 모델이 다른 프레임워클 훈련되어 있다면 수동으로 모델 파라미터를 읽어 들여 적절한 변수에 할당해야 한다. 

In [0]:
#원본모델이 tensorflow로 학습되었으면 바로 불러와서 사용가능
#import_meta_graph로 기존 연산에 적재한다. 
reset_graph()
original_w=[[1.,2.,3.], [4.,5.,6.]] #기존의 가중치
original_b=[7.,8.,9.] #기존의 bias

X=tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
hidden1=tf.layers.dense(X,n_hidden1, activation=tf.nn.relu, name="hidden1",reuse=tf.AUTO_REUSE)

graph=tf.get_default_graph()
assign_kernel=graph.get_operation_by_name("hidden1/kernel/Assign") #선언한 노드이름으로 불러오는 과정
assign_bias=graph.get_operation_by_name("hidden1/bias/Assign")
init_kernel= assign_kernel.inputs[1]
init_bias=assign_bias.inputs[1]

init=tf.global_variables_initializer() #graph변수 초기화 

```python
#graph실행 세션
with tf.Session() as sess:
  sess.run(init, feed_dict={init_kernel:original_w, init_bias: original_b})
  print(hidden1.eval(feed_dict={X:[[10.0, 11.0]]})) #각 세션 layer ReLU(weight+bias)결과 확인
```

In [35]:
tf.get_default_graph().get_tensor_by_name("hidden1/kernel:0")

<tf.Tensor 'hidden1/kernel:0' shape=(784, 300) dtype=float32_ref>

In [31]:
tf.get_default_graph().get_tensor_by_name("hidden1/bias:0")

<tf.Tensor 'hidden1/bias:0' shape=(300,) dtype=float32_ref>

#### 11.2.3. 신경망의 하위층을 학습에서 제외하기 
- 첫번째 DNN의 하위층은 이미 저수준의 특성을 감지하도록 학습되어 있기 때문에 다른 이미지 분류 작업에 유용하다
- 일반적으로 새로운 DNN을 훈련시킬 때 재사용되는 층들의 가중치를 동결하는 것이 좋다. 

```python
#은닉층 3,4층과 출력층에 있는 학습할 변수 목록 저장
train_vars=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="hidden[34]|outputs")
#훈련에 사용할 변수를 동결하는 방법 1
training_op=optimizer.minimize(loss, var_list=train_vars)
#훈련데 사용할 변수를 동결하는 방법 2
with tf.name_scope("dnn"):
  hidden1=tf.layers.dense(X,n_hidden1, activation=tf.nn.relu, name="hidden")
  hidden2=tf.layers.dense(hidden1, n_hidden2, activation=tf.nn.relu, name="hidden2")
  hidden3=tf.layers.dense(hidden2_stop, n_hidden3, activation=tf.nn.relu, name="hidden3")
  hidden4=tf.layers.dense(hidden3, n_hidden4, activation=tf.nn.relu, nam="hidden4)
  logits=tf.layers.dense(hidden4, n_outputs, name="outputs")
```

#### 11.2.4. 동결된 층 캐싱하기 
- 캐싱 : 일반적인 특징이 있는 데이터의 하위 집합을 저장하는 고속 데이터 스토리지를 의미한다. 
- 동결된 층은 변하지 않기 때문에 각 훈련 샘플에 대해 가장 위쪽의 동결된 층에서 나온 출력을 캐싱하는 것이 가능하다. 
- 이렇게 만들어 버리면 동결된 층을 지날 때 학습속도가 훨씬 좋아진다. 

```python
import numpy as np

n_batches=len(X_train) #batch_size
with tf.Session() as sess:
  init.run()
  restore_saver.restore(sess, './my_model_final.ckpt')
  h2_cache=sess.run(hidden2, feed_dict={X:x_train})

  for epoch in range(n_epochs):
    shuffled_idx=np.random.permutation(len(X_train))
    hidden2_batches=np.array_split(h2_cache[shuffled_idx], n_batches)
    y_batches=np.array_split(y_train[shuffled_idx], n_batches)
    for hidden2_batch, y_batch in zip(hidden2_batches, y_batches):
      sess.run(training_op, feed_dict={hidden2:hidden2_batch, y:y_batch})
  save_path=saver.save(sess, ",/my_new_model_final.ckpt")
#두번째 은닉층의 출력을 배치로 만들어 주입하는 경우이다. 
```

#### 11.2.5. 상위층을 변경, 삭제, 대체하기 
- 원본모델에서의 상위층은 대체로 덜 유용하므로 거의 대체된다고 본다. 
- 그래서 재사용할 적절한 층의 개수를 아는 것이 중요하다
- **적절한 재사용층 아는 방법** : 
  1. 복사한 층을 동결 후 훈련하여 성능을 평가 
  2. 위쪽의 은닉층을 동결 해제하면서 가중치 변경하여 성능이 좋아지는지를 확인
  3. 그래도 좋은 성능을 얻지 못하면 가장 위쪽의 은닉층을 제거하고 남은 은닉층을 동결한다. 

#### 11.2.6. 모델 저장소
- 당면한 문제와 비슷한 작업을 훈련시킨 신경망을 찾을 수 있는 곳
  1. 자신의 모델 카탈로그
  2. 텐서플로우에서 제공하는 모델 저장소 : https://github.com/tensorflow/models
  3. 카페 모델 저장소 : https://goo.gl/XI02X3


#### 11.2.7. 비지도 사전훈련
- 훈련 데이터가 부족한 경우에 전이학습을 통해 해결한다고 앞에서 언급했음. 하지만 비슷한 작업에 대한 훈련된 모델을 찾을 수가 없다면?
  1. 더 많은 훈련 데이터를 찾는다
  2. 비지도 사전훈련을 하는 경우 : 오토인코더, 제한된 볼트만 머신
- 모든 층이 훈련되면 지도학습으로 신경망을 세밀하게 튜닝이 가능하다. (375p그림)

#### 11.2.8. 보조작업으로 사전훈련
- 레이블된 훈련 데이터를 쉽게 얻거나 생성할 수 있는 보조 작업에 첫번째 신경망을 훈련하는 것이다. 
- 신경망의 하위층을 실제 작업을 위해 재사용하는 것이다. 
- ex. image detection   : 하나의 이미지로 iteration에 따라 다르게 학습시키는 경우 
